# Tarea 01: Corrector gramatical
Enrique Ulises Báez Gómez Tagle
Mauricio Iván Ascencio Martínez
Sara Rocio Miranda Mateos

### Para esta tarea, deberás de generar un corrector gramatical que evalué 5 tipos de errores comunes, llenando los métodos que aparecen en este notebook. Deberás de utilizar la lógica adecuada para cada uno y usar el formato de docstring correspondiente para documentar el uso de cada método.

### ¿Qué es un corrector gramatical?

Es un sistema de revisión de textos, capaz de implementar reglas de análisis y corrección de textos basados en el idioma específico con el que se está trabajando.

Capacidad de corrección:

Entre otras cosas, un correcto gramatical puede detectar lo siguiente:

- Faltas de ortografía en palabras comunes  (Por fabor  →  Por favor) 

- Combinación incorrecta de singulares y plurales  (Mi casas  →  Mi casa)

- Palabras repetidas (Le dije eso eso ayer  →  Le dije eso ayer)

- Mayúsculas en lugares incorrectos  (LA tarDe  →  La tarde)

- Problemas con verbos auxiliares (Debes pagado  →  Debes pagar)

Entre otras aplicaciones

In [10]:
# # Ejemplo de formato Docstrings:
# 
# def NombreFuncion(arg1, arg2, arg3):
#   """
#   Este método sirve para... utilizando... y devuelve...
# 
#   Args:
#     string arg1: Esta es una cadena de texto que...
#     int arg 2: Es un número entero que se usa para...
#     dict arg 3: Diccionario que sirve para...
# 
#   Returns:
#     string: Cadena del texto ya corregido...
#   """
# 
#   # Aquí debe de ir la lógica de la función (Después de la documentación)
#   Texto = ""
#   return Texto

#### Plantilla de tarea 1: Corrector gramatical

In [11]:
# Importar la librería de SpaCy y su núcleo de trabajo en Español
import spacy
from spacy import displacy

# Cargamos núcleo de trabajo (Español)
nlp = spacy.load("es_core_news_lg")

In [12]:
# Módulo para correguir errores de ortografía comunes
# Utilizando un diccionario
# Agregar mas ejemplos al diccionario

Diccionario = {
    "q": "que",
    "ola": "hola",
    "zaldo": "saldo",
    "fabor": "favor",
    "grasias": "gracias",
    "x": "por",
    "cmo": "cómo",
    "dnd": "dónde",
    "sq": "es que",
    "sta": "esta",
    "msj": "mensaje",
    "aki": "aquí",
    "tmb": "también",
    "qro": "quiero",
    "pq": "porque",
}


def errores_comunes(texto):
    """Esta función verifica la ortografía de las palabras en un texto y corrige errores.

    Args:
        string Texto: El texto que se va a revisar.

    Returns:
        string Texto_corregido: El texto con los errores corregidos.
    """
    doc = nlp(texto)
    texto_corregido = ""
    for token in doc:
        if token.text.lower() in Diccionario:
            # REPLACE WORD WITH CORRECT ONE
            texto_corregido += Diccionario[token.text.lower()]
        else:
            texto_corregido += token.text
        texto_corregido += " "
    return texto_corregido

In [13]:
# Módulo para correguir errores dentre singulares y plurales
# Investigar si existe algún atributo en Spacy que pueda devolver dicha característica
# O implementar alguna lógica que pueda detectar si una palabra es sing, o plural


def correct_number_mismatches(doc):
    """Esta función verifica que no exista una palabra en singular, seguida de una en plural y al revés.

    Args:
        string doc: El texto que se va a revisar.

    Returns:
        list correcciones: La lista de correcciones sugeridas
    """
    correcciones = []
    for i, token in enumerate(doc[:-1]):
        next_tok = doc[i + 1]

        if (
                token.tag_ == "DET"
                and token.text[-1] == "s"
                and next_tok.tag_ == "NOUN"
                and next_tok.text[-1] != "s"
        ):
            correcciones.append((next_tok.text, next_tok.text + "s"))
        elif (
                token.tag_ == "DET"
                and token.text[-1] != "s"
                and next_tok.tag_ == "NOUN"
                and next_tok.text[-1] == "s"
        ):
            correcciones.append((next_tok.text, next_tok.text[:-1]))

    return correcciones


def apply_corrections(text, correcciones):
    """Esta función aplica las correcciones sugeridas por la función anterior.

    Args:
        string text: El texto que se va a revisar.
        list correcciones: La lista de correcciones sugeridas.

    Returns:
        string text: El texto con los errores corregidos.
    """
    for original, corrected in correcciones:
        text = text.replace(original, corrected)
    return text


def tiempos_verbales(texto):
    """Esta función corrige errores en los tiempos verbales usando las 2 funciones auxiliares definidas anteriormente.

    Args:
        texto: El texto que se va a revisar.

    Returns:
        El texto con los errores corregidos.
    """
    doc = nlp(texto)
    correcciones = correct_number_mismatches(doc)
    texto_corregido = apply_corrections(texto, correcciones)

    return texto_corregido

In [14]:
# Módulo para detectar palabras repetidas


def palabra_repetida(texto):
    """Esta función verifica que no exista la misma palabra repetida dos veces seguidas en un texto.

    Args:
        string texto: El texto a revisar.

    Returns:
        string texto_corregido: El texto corregido.
    """
    doc = nlp(texto)

    texto_corregido = ""

    prev_token = None
    for token in doc:
        if prev_token is not None and token.text.lower() == prev_token.text.lower():
            continue
        else:
            texto_corregido += token.text + " "
            prev_token = token

    return texto_corregido.strip()

In [15]:
# Módulo para detectar problemas con mayúsculas y minúsculas
# Recuerda que si una palabra son SIGLAS (Es decir, se trata de una organización
# o lugar), NO se considera un error gramatical
def check_token_case(token):
    """Verifica si el token cumple con la regla de mayúsculas y minúsculas.

    Args:
        token (Token): El token a verificar.

    Returns:
        bool: True si el token cumple con la regla, False si no.
    """
    text = token.text

    if text.isupper():
        return True
    elif text.islower():
        return True
    elif text.isdigit():
        return True
    else:
        if (any(char.islower() for char in text) and any(char.isupper() for char in text)) or \
                (any(char.islower() for char in text) and any(char.isdigit() for char in text)) or \
                (any(char.isdigit() for char in text) and any(char.isupper() for char in text)):
            return False
    return True


def correct_token_case(text):
    """Corrige los tokens del texto que no cumplan con la regla de mayúsculas y minúsculas.

    Args:
        text (str): El texto a corregir.

    Returns:
        str: El texto corregido.
    """
    doc = nlp(text)
    corrected_tokens = []

    for token in doc:
        if not check_token_case(token):
            corrected_tokens.append(token.text.lower())
        else:
            corrected_tokens.append(token.text)

    return " ".join(corrected_tokens)


def capitalize_after_period(text):
    """Corrige un texto para que después de un punto y la primera palabra empiecen con mayúscula.

    Args:
        string text: El texto a corregir.

    Returns:
        str: El texto corregido.
    """
    # Primero, aseguramos que la primera palabra esté en mayúsculas
    text = text[0].upper() + text[1:]

    # Luego, aseguramos que cualquier palabra después de un punto esté en mayúsculas
    sentences = text.split(". ")
    sentences = [s[0].upper() + s[1:] if len(s) > 0 else s for s in sentences]

    return ". ".join(sentences)


def mayusc_minusc(texto):
    """ Esta funcion se apoya en las 3 funciones auxiliares definidas anteriormente para asegurar el correcto uso de las mayúsculas y minúsculas

    Args:
        string texto: El texto a revisar.

    Returns:
        string texto: El texto revisado.
    """
    texto_corregido = capitalize_after_period(correct_token_case(texto))
    return texto_corregido

In [16]:
# Modulo para detectar nouns juntos que no son un nombre propio
# Recuerda que "Juega el perro gato en el parque" es incorrecto
# porque hay dos NOUNS juntos (perro y gato), pero el texto "mi
# amigo Pedro" es correcto, sin embargo aquí se trata de un NOUN
# seguido de un PNOUN (Nombre propio), deberás de identificar si
# si ocurre un caso como este, o si por ejemplo, todo el nombre
# pertenece a una persona (entidad) para validar que no sea un error


def check_noun_noun_rule(text):
    """Esta función verifica que no exista dos sustantivos seguidos que no sean nombres propios.

    Args:
        text: El texto a revisar.

    Returns:
        True si no hay dos sustantivos seguidos que no sean nombres propios, False si sí hay.
    """
    doc = nlp(text)
    for i, token in enumerate(doc[:-1]):
        next_tok = doc[i + 1]

        if (
                (token.tag_ == "PROPN" or token.tag_ == "NOUN")
                and (next_tok.tag_ == "PROPN" or next_tok.tag_ == "NOUN")
                and not next_tok.text.istitle()
        ):
            return False
    return True


def detectar_dos_nouns_seguidos(texto):
    """Esta función verifica que no exista dos sustantivos seguidos que no sean nombres propios.

    Args:
        string texto: El texto a revisar.

    Returns:
        Una lista de los segundos sustantivos de los pares encontrados y el texto corregido.
    """
    if not check_noun_noun_rule(texto):
        doc = nlp(texto)

        pares_sustantivos = []

        token_ant = None
        for token in doc:
            if (
                    token_ant is not None
                    and (token_ant.tag_ == "PROPN" or token_ant.tag_ == "NOUN")
                    and (token.tag_ == "PROPN" or token.tag_ == "NOUN")
                    and not token.text.istitle()
            ):
                pares_sustantivos.append((token_ant.i, token.i))
            token_ant = token

        segundos_sustantivos = [doc[pair[1]].text for pair in pares_sustantivos]

        texto_corregido = []
        skip_next = False
        for i, token in enumerate(doc):
            if any([pair[1] == i for pair in pares_sustantivos]):
                skip_next = True
            if skip_next:
                skip_next = False
                continue
            texto_corregido.append(token.text)

        return segundos_sustantivos, " ".join(texto_corregido)

    else:
        return [], texto


def nouns(texto):
    """Esta función se apoya en las 2 funciones auxiliares definidas anteriormente para asegurar que no haya 2 nouns juntos
    Args:
        texto: texto a analizar
    Returns:
        texto_corregido: texto corregido si hay 2 nouns juntos, en otro caso devuelve el mismo texto
    """
    segundos_sustantivos, texto_corregido = detectar_dos_nouns_seguidos(texto)
    if segundos_sustantivos:
        return texto_corregido
    else:
        return texto

In [17]:
# El siguiente código debe de poder ser ejecutado y obtener una respuesta que
# muestre cláramente la diferencia entre el texto inicial y el corregido

# Cada método deberá marcar el lugar donde encuentre un error con corchetes
# e imprimir una lista con los errores encontrados

Utterance = input("texto para revisión: \n")

# Pipeline de funciones definidas previamente
texto_corregido = nouns(mayusc_minusc(palabra_repetida(tiempos_verbales(errores_comunes(Utterance)))))

print('\ntexto corregido:\n' + texto_corregido)


texto corregido:
Hola , me puede dar mi saldo por favor , gracias , lo que pasa es que quiero comprar una casa en la playa para que los niños jueguen en el patio . Mi perro es muy juguetón , por eso me mudo a la CDMX . El casero me dijo : " debes pagarlo antes del viernes " .


In [18]:
# FRASE DE PRUEBA
# ola señorita, me puede dar mi zaldo por fabor, grasias, lo que pasa pasa es que quiero Comprar una casas en la playa playa para que los niño jueguen en el patio. Mi PeRRo es muy juguetón, por eso me mudo a la CDMX. El casero mE dijo: "debes pagarlo antes del viernes viernes".

# RESULTADO OBTENIDO
# texto corregido:
# Hola , me puede dar mi saldo por favor , gracias , lo que pasa es que quiero comprar una casa en la playa para que los niños jueguen en el patio . Mi perro es muy juguetón , por eso me mudo a la CDMX . El casero me dijo : " debes pagarlo antes del viernes " .
